In [ ]:
from functools import partial

import torch
import torch.nn as nn

import get_mixed_data

import importlib
importlib.reload(get_mixed_data)


from preprocess import pad_collate_fn

import training
importlib.reload(training)
from training import get_train_val_loader, ClassifierLearner

import model

from model import FinalModel

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

In [ ]:
(index_to_word, word_to_index,
     dict_wiki_tensor_dataset,
     weights_matrix_ve, classes) = get_mixed_data.get_datasets(LANGUAGES_LIST=["english", "russian", "hindi"])

In [32]:
train_loader, val_loader = get_train_val_loader(
    dict_wiki_tensor_dataset["train"], dict_wiki_tensor_dataset["val"], 
    collate_fn=partial(pad_collate_fn, pad_token=word_to_index["<pad>"])
)

In [74]:
SAVE_MODEL = False

batch_size = 8
lr = 0.01
num_epochs = 15

options = {
    "VOCAB_SIZE": len(index_to_word),
    "dim_e": weights_matrix_ve.shape[1],
    "pretrained_embeddings": weights_matrix_ve,
    "num_layers": 2,
    "num_classes": len(classes),
    "mid_features": 150,
    "dropout_rate": 0.2,
    "activation": nn.ReLU(),
}
    
result = {
    "optimizer": "Adam", 
    "num_hidden": options["num_layers"],
    "dim_hidden": options["mid_features"],
    "dropout_rate": options["dropout_rate"],
    "learning_rate": lr,
    "num_epochs": num_epochs
}


print("\n", result)
model_name = "mixed_en_hi_ru_" + "_".join([str(key) + "_" + str(value) for key, value in result.items()])

learner = ClassifierLearner(options, model_name, device=device)
learner.set_loaders(train_loader, val_loader)


 {'optimizer': 'Adam', 'num_hidden': 2, 'dim_hidden': 150, 'dropout_rate': 0.2, 'learning_rate': 0.01, 'num_epochs': 15}


In [76]:
learner.train_model(num_epochs=2, lr=0.01)

0 epoch
Epoch: [1/2], Step: [101/1250], Train_loss: 0.05512901475653052
Epoch: [1/2], Step: [201/1250], Train_loss: 0.057675051661208274
Epoch: [1/2], Step: [301/1250], Train_loss: 0.05829053848360976
Epoch: [1/2], Step: [401/1250], Train_loss: 0.0579481419804506
Epoch: [1/2], Step: [501/1250], Train_loss: 0.057547365326434376
Epoch: [1/2], Step: [601/1250], Train_loss: 0.05717676502652466
Epoch: [1/2], Step: [701/1250], Train_loss: 0.05742283483701093
Epoch: [1/2], Step: [801/1250], Train_loss: 0.05803204459603876
Precision macro: 0.6984, Recall macro: 0.4459, F1 macro: 0.521 
Precision micro: 0.8374, Recall micro: 0.6314, F1 micro: 0.7199 
Epoch: [1/2], Step: [901/1250], Train_loss: 0.058117103338655496
Epoch: [1/2], Step: [1001/1250], Train_loss: 0.05853012042865157
Epoch: [1/2], Step: [1101/1250], Train_loss: 0.059197065472941506
Epoch: [1/2], Step: [1201/1250], Train_loss: 0.05928395319730043
1 epoch
Epoch: [2/2], Step: [101/1250], Train_loss: 0.05721411112695932
Epoch: [2/2], Ste

({'precision_macro': 0.7130980757134329,
  'recall_macro': 0.5045685755680909,
  'f1_macro': 0.5651468087958444,
  'precision_micro': 0.8265726043503822,
  'recall_micro': 0.6679334916864608,
  'f1_micro': 0.7388334209143458},
 1)

In [51]:
len(learner.val_loader.dataset)

1000

In [53]:
from torch.utils import data

In [77]:
learner.get_test_metrics_kfold(3, learner.val_loader.dataset)

/scratch/mz2476/miniconda3/envs/my_base/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/mz2476/miniconda3/envs/my_base/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,precision_macro,recall_macro,f1_macro,precision_micro,recall_micro,f1_micro
0,0.601943,0.474778,0.501424,0.806897,0.659155,0.725581
1,0.581520,0.475961,0.501708,0.798276,0.685926,0.737849
2,0.601665,0.457821,0.497679,0.795041,0.668056,0.726038


In [55]:
import numpy as np

In [69]:
[np.arange(start, 100, num_splits) for start in range(num_splits)]

[array([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80,
        85, 90, 95]),
 array([ 1,  6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56, 61, 66, 71, 76, 81,
        86, 91, 96]),
 array([ 2,  7, 12, 17, 22, 27, 32, 37, 42, 47, 52, 57, 62, 67, 72, 77, 82,
        87, 92, 97]),
 array([ 3,  8, 13, 18, 23, 28, 33, 38, 43, 48, 53, 58, 63, 68, 73, 78, 83,
        88, 93, 98]),
 array([ 4,  9, 14, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, 69, 74, 79, 84,
        89, 94, 99])]

In [64]:
indices = np.arange(500)
indices = np.arange(500, 1000)

dict_of_metrics = learner.get_test_metrics(
    data.Subset(learner.val_loader.dataset, indices), device=learner.device)

import pandas as pd

pd.DataFrame([dict_of_metrics, dict_of_metrics])

/scratch/mz2476/miniconda3/envs/my_base/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/scratch/mz2476/miniconda3/envs/my_base/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,precision_macro,recall_macro,f1_macro,precision_micro,recall_micro,f1_micro
0,0.651972,0.482795,0.534985,0.798883,0.680304,0.734841
1,0.651972,0.482795,0.534985,0.798883,0.680304,0.734841


In [ ]:
# train the model
model_name = "mixed_en_hi_ru_" + "_".join([str(key) + "_" + str(value) for key, value in result.items()])
print(model_name)
metrics_dict = train_model(train_loader, val_loader, model, criterion, optimizer, options, device,
                num_epochs=10, model_name="model", save_model=False)
result.update(metrics_dict)

In [8]:
optimizer.param_groups[0]["lr"]

0.01

In [19]:
# loaders = create_data_loaders_for_model(wiki_loaders["train"], wiki_loaders["val"])
# # create dataloader
# wiki_loaders = {}


# for split, wiki_dataset in dict_wiki_tensor_dataset.items():
#     wiki_loaders[split] = DataLoader(
#         wiki_dataset, 
#         batch_size=batch_size, 
#         shuffle=True, 
#         collate_fn=partial(pad_collate_fn, word_to_index=word_to_index)
#     )


# train_model(
#     wiki_loaders, model, criterion, optimizer, options=options, num_epochs=num_epochs, 
#     model_name=model_name, save_model=SAVE_MODEL
# )

# results_df = results_df.append(result, ignore_index=True)
#     results_df.to_csv("results/results_tuning_2_3_layers_maxlen_500.csv"


 {'optimizer': 'Adam', 'num_hidden': 2, 'dim_hidden': 150, 'dropout_rate': 0.2, 'learning_rate': 0.01, 'num_epochs': 15}


RuntimeError: CUDA out of memory. Tried to allocate 2.28 GiB (GPU 0; 7.93 GiB total capacity; 6.85 GiB already allocated; 626.56 MiB free; 6.49 MiB cached)